# Los Angeles Behavioral Incentives Case Study (USC Focus)\n\n**Policy:** Measure ULA (Mansion Tax)  \n**Policy date:** 2023-04-01  \n**Economic series:** FRED `ATNHPIUS31080Q` (LA-Long Beach-Anaheim metro house price index)  \n**Sentiment source:** Reddit discussions on student loan policy\n

In [ ]:
import pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom IPython.display import Markdown, display\n\nsns.set_theme(style='whitegrid')\npolicy_date = pd.to_datetime('2023-04-01')\n

In [ ]:
monthly = pd.read_csv('../reports/monthly_series.csv')\nsummary = pd.read_csv('../reports/policy_summary.csv')\nsentiment = pd.read_csv('../data/processed/reddit_sentiment.csv')\n\nif 'created_utc' in sentiment.columns:\n    sentiment['created_utc'] = pd.to_datetime(sentiment['created_utc'])\n    sentiment_daily = (\n        sentiment.groupby(sentiment['created_utc'].dt.date, as_index=False)\n        .agg(avg_compound=('compound', 'mean'), posts=('id', 'count'))\n    )\nelse:\n    sentiment_daily = pd.DataFrame(columns=['created_utc', 'avg_compound', 'posts'])\n\nmonthly['month'] = pd.to_datetime(monthly['month'])\n

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))\nsns.lineplot(data=monthly, x='month', y='monthly_avg_value', marker='o', ax=ax)\nax.axvline(policy_date, color='red', linestyle='--', label='Policy Start')\nax.set_title('LA Metro House Price Index Around Measure ULA')\nax.set_xlabel('Date')\nax.set_ylabel('Index')\nax.legend()\nplt.tight_layout()\nplt.show()\n

In [ ]:
if not sentiment_daily.empty:\n    fig, ax1 = plt.subplots(figsize=(11, 4))\n    sns.lineplot(data=sentiment_daily, x='created_utc', y='avg_compound', marker='o', ax=ax1, color='#1f77b4')\n    ax1.set_ylabel('Avg Sentiment (Compound)')\n    ax1.set_xlabel('Date')\n\n    ax2 = ax1.twinx()\n    sns.barplot(data=sentiment_daily, x='created_utc', y='posts', alpha=0.25, ax=ax2, color='#ff7f0e')\n    ax2.set_ylabel('Post Count')\n    ax2.grid(False)\n\n    ax1.set_title('Sentiment + Discussion Volume (Student Loan Topic)')\n    plt.tight_layout()\n    plt.show()\nelse:\n    print('Sentiment file missing expected columns or no rows.')\n

In [ ]:
pre = summary.loc[summary['metric'] == 'pre_policy_avg', 'value']\npost = summary.loc[summary['metric'] == 'post_policy_avg', 'value']\npct = summary.loc[summary['metric'] == 'percent_change', 'value']\n\npre_val = float(pre.iloc[0]) if len(pre) else float('nan')\npost_val = float(post.iloc[0]) if len(post) else float('nan')\npct_val = float(pct.iloc[0]) if len(pct) else float('nan')\n\nsent_n = int(len(sentiment))\nsent_avg = float(sentiment['compound'].mean()) if 'compound' in sentiment.columns and len(sentiment) else float('nan')\n\nmemo = f'''## Admissions-Ready Interpretation Draft\n\n**Research question:** Do behavioral expectations seen in social discourse provide useful context for market shifts around policy change in Los Angeles?\n\n**Economic finding (LA housing index):** The average index moved from **{pre_val:.2f}** before Measure ULA (2023-04-01) to **{post_val:.2f}** after, a change of **{pct_val:.2f}%**.\n\n**Behavioral signal finding (social discourse):** Across **{sent_n}** posts on student-loan policy, average sentiment was **{sent_avg:.3f}**. This captures expectation, uncertainty, and perceived fairness, which are key behavioral incentives.\n\n**Interpretation:** Policy does not only alter constraints; it changes beliefs and strategic behavior. Sentiment and engagement can serve as lead indicators for shifts in consumer confidence and decision-making.\n\n**Limitations:** This is exploratory (not causal identification), social media samples are non-random, and LA metro index reflects broader forces beyond one policy.\n\n**Next step for rigor:** Add interrupted time-series or difference-in-differences with a control metro to strengthen causal claims.\n'''\ndisplay(Markdown(memo))\n